In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import random

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import yfinance as yf
import requests

In [3]:
df = pd.read_csv("https://query.data.world/s/pzd7uazpmpyortmg6fsuxiaoizllgu?dws=00000", encoding='ISO-8859-1')
display(df.head())

,Public Company Name (As Listed on Exchange),B Corp Impact Data ID,Ticker 1,Ticker 2,Ticker 3,IPO Year,Year Delisted,Year Certified,Public Listing Status,Year Decertified
0,"Allbirds, Inc.",001C000001SGhnyIAD,NASDAQGS:BIRD,NaN,NaN,2021.0,NaN,2016.0,active,NaN
1,Amalgamated Financial,001C000001K16JSIAZ,NASDAQGM:AMAL,NaN,NaN,2018.0,NaN,2017.0,active,NaN
2,"AppHarvest, Inc.",0013b00001o41JFAAY,NASDAQGS:APPH,NaN,NaN,2020.0,NaN,2019.0,delisted,2023.0
3,Arezzo&Co,0013b00001qHFddAAG,BOVESPA:ARZZ3,NaN,NaN,2011.0,NaN,2022.0,active,NaN
4,Arowana International,001C000001ZzPK4IAN,ASX: AWN,NaN,NaN,2013.0,2021.0,2018.0,delisted,NaN


In [4]:
df.drop(['B Corp Impact Data ID', 'Ticker 2', 'Ticker 3'], axis=1, inplace = True)
print(len(df))
df.dropna(axis=0, subset=['Public Company Name (As Listed on Exchange)'], inplace=True)
print(len(df))

999
68


In [5]:
display(df[df['Public Listing Status'] == 'delisted'])

,Public Company Name (As Listed on Exchange),Ticker 1,IPO Year,Year Delisted,Year Certified,Public Listing Status,Year Decertified
2,"AppHarvest, Inc.",NASDAQGS:APPH,2020.0,NaN,2019.0,delisted,2023.0
4,Arowana International,ASX: AWN,2013.0,2021.0,2018.0,delisted,NaN
14,Freshlocal Solutions,TSE: LOCL,2019.0,2021.0,2013.0,delisted,NaN


In [6]:
temp = df.copy()
temp = temp[temp['Public Listing Status'] == 'active']

temp['Ticker 1'].replace(": ", ":", regex=True, inplace=True)


print(len(temp))
temp['Exchange'] = temp['Ticker 1'].str.split(":").str[0]
temp['Ticker2'] = temp['Ticker 1'].str.split(":").str[1]
temp.dropna(subset=["Ticker2"], inplace=True) 

#display(temp)

65


In [7]:
#ticks = temp['Ticker2'].to_list()
#for i in range(len(ticks)):
#    print(ticks[i], type(ticks[i]), i)

In [8]:
#Find datafeed issues

#for t in ticks:
#    ticker = yf.Ticker(t)
#    info = ticker.info
#    info = None

#    try:
#        print("trying")
#        info = ticker.info
#    except:
#        print("except")
        #print(f"Cannot get info of {t}, it probably does not exist")
#    continue

  # Got the info of the ticker, do more stuff with it
    #print(f"Info of {t}: {info}")


In [9]:
#yf.Ticker('BFT.WA').info
#yf.Ticker('LFG.XA').info

In [10]:
exchange_dict = {'ASX': '.XA', 
                 'WSE': '.WA',
                 'BOVESPA': '.SA',
                 'ETHEX': '',
                 'TSX': '.TO',
                 'ENXTPA': '.PA',
                 'BME': '.MC',
                 'ENXTBR': '.BR',
                 'KLSE': '.KL',
                 'LSE': '.AQ',
                 'TWSE': '.TW',
                 'NZSE': '.NZ',
                 'SGX': '.SI',
                 'BIT': '.MI',
                 'TASE': '.TA',
                 'XMIL': '.MI',
                 'NZE': '.NZ',
                 'BVMF': '.SA',
                 'HKG': '.HK',
                 'LON': '.L',
                 'MIL': '.MI',
                 'MOVI3': '.SA',
                 'TSE': '.T',
                 'BRK': '.L',
                 'SNSE': '.SN',
                 'NYSE': '',
                 'NASDAQGS': '',
                 'NASDAQGM': '',
                 'NASDAQCM': ''
                }

exchange_map = {exchanges: extention
                 for exchanges, extention in exchange_dict.items()
                 for exchange in exchanges}

temp['Intl Extention'] = temp['Exchange'].map(exchange_map)

In [11]:
temp['Ticker'] = temp['Ticker2'] + temp['Intl Extention'] 
mask = temp['Ticker'].isna()
temp[mask]

,Public Company Name (As Listed on Exchange),Ticker 1,IPO Year,Year Delisted,Year Certified,Public Listing Status,Year Decertified,Exchange,Ticker2,Intl Extention,Ticker
19,Heidelberg District Community Enterprise Limited,NSX:EIC,2019.0,NaN,2019.0,active,NaN,NSX,EIC,NaN,NaN
29,Movida Aluguel de Carros,MOVIDA B3:MOVI3,2017.0,NaN,2020.0,active,NaN,MOVIDA B3,MOVI3,NaN,NaN
36,SIDERPERU,BVL:SIDERC1,2006.0,NaN,2022.0,active,NaN,BVL,SIDERC1,NaN,NaN
41,Urb-it AB (publ),OM:URBIT,2017.0,NaN,2020.0,active,NaN,OM,URBIT,NaN,NaN
48,Weleda AG Group,OTC-X:BEKB:Weleda N-PS,NaN,NaN,2021.0,active,NaN,OTC-X,BEKB,NaN,NaN
56,Vinedos Emiliana S.A.,XSGO:EMILIANA,1996.0,NaN,2023.0,active,NaN,XSGO,EMILIANA,NaN,NaN


In [12]:
#Find datafeed issues
tickers = temp['Ticker'].to_list()

count = 0
for t in tickers:
    try:
        ticker = yf.Ticker(t)
        info = None
    except:
        count +=1
        print(f"Cannot get info of {t}, it probably does not exist")

print("Total nans =", count, "which is", round(count/temp.shape[0],2)*100, '% of the total')



Cannot get info of nan, it probably does not exist
Cannot get info of nan, it probably does not exist
Cannot get info of nan, it probably does not exist
Cannot get info of nan, it probably does not exist
Cannot get info of nan, it probably does not exist
Cannot get info of nan, it probably does not exist
Total nans = 6 which is 10.0 % of the total


In [13]:
# Remove Nans from stock universe:

b_corp_df = temp[-mask].copy()
display(b_corp_df.head())


,Public Company Name (As Listed on Exchange),Ticker 1,IPO Year,Year Delisted,Year Certified,Public Listing Status,Year Decertified,Exchange,Ticker2,Intl Extention,Ticker
0,"Allbirds, Inc.",NASDAQGS:BIRD,2021.0,NaN,2016.0,active,NaN,NASDAQGS,BIRD,,BIRD
1,Amalgamated Financial,NASDAQGM:AMAL,2018.0,NaN,2017.0,active,NaN,NASDAQGM,AMAL,,AMAL
3,Arezzo&Co,BOVESPA:ARZZ3,2011.0,NaN,2022.0,active,NaN,BOVESPA,ARZZ3,.SA,ARZZ3.SA
5,Australian Ethical Investment Ltd.,ASX:AEF,2002.0,NaN,2014.0,active,NaN,ASX,AEF,.XA,AEF.XA
6,Benefit Systems S.A.,WSE:BFT,2013.0,NaN,2018.0,active,NaN,WSE,BFT,.WA,BFT.WA


In [14]:
isolate_us = False
if isolate_us:

    exchanges = set(temp['Exchange'])
    print(exchanges)
    us_ex = ['NYSE', 'NASDAQGS','NASDAQGM', 'NASDAQCM']
    us = temp[temp['Exchange'].isin(us_ex)]
    print("Isolating US Companies")
    display(us.head())


### List of Companies with B-Corp Subsidiaries
Unilever (NYSE: UN): (Pukka (UK), Mãe Terra (Brazil), Sir Kensington’s (US), Ben & Jerry’s (US), Olly Nutrition (US), Seventh Generation (US), Sundial (US), T2 Tea (Australia), Graze (UK))

B2W (B3: BTOW3): Courrieros (Brazil)

Grupo Soma (B3: SOMA3): Companhia Hering (Brazil)

Arezzo&Co (B3: ARZZ3): Reserva (Brazil)

Procter & Gamble (NYSE: PG): (New Chapter (US))

AB Inbev: (4 Pines Brewing Company)

Banco Estado: (Banco Estado Microempresas, Caja Vecina)

Coca-Cola: (Innocent Drinks)

Fairfax Financial: (The Redwoods Group)

Gap: (Athleta, Hill City)

Kikkoman: (Country Life)

Lactalis: (Stonyfield Farm)

Land O’ Lakes: (Vermonta Creamery)

Nestle: (Essential Living Foods, Garden of Life, Lily's Kitchen)

Oppenheimer Funds Inc.: (SNW Asset Management)

Rakuten: (OverDrive)

The Hain Celestial Group: (Ella’s Kitchen)

Vina Concha y Toro: (Fetzer Vineyards)


Hortfruti S.A.: Hortifruti Chile

Danone SA: (Danone North America, Alpro, Danone Waters of America, Danone Canada, -Aguas Danone Argentina, Bledina, Danone Aqua Indonesia, Danone Egypt, Danone Iberia, Danone Manifesto Ventures, Danone UK, Grameen Danone Foods, Happy Family, Les 2 Vaches, Danone Waters Germany, Danone Dairy Ireland, Danone Waters Spain, Danone Netherlands, Danone Dairy Belgium, Nutricia Bago, Danone ELN Greater China, Danone Japan, Volvic, Danone Brazil)

Caixabank (Imagin Ban)

Banco Sabadell (Sogeviso)

iSun (SunCommon)

In [15]:
add_subs = True
public_subs = ['UL', 'SOMA3.SA', 'ABI.BR', 'PG', 'FFH.TO', 'BUD', 'KO', 'GPS', 'KIK.F', 'NESN.SW',
               'OPY', '4755.T', 'HAIN', 'DANOY', 'CABK.MC', 'ISUN']

In [16]:
def get_stock_data (df):
    tickers_list = df['Ticker'].tolist()
    if add_subs:
        tickers_list = tickers_list + public_subs
    tickers_data= {}
    
    for ticker in tickers_list:
        ticker_object = yf.Ticker(ticker)
        #convert info() output from dictionary to dataframe
        temp = pd.DataFrame.from_dict(ticker_object.info, orient="index")
        temp.reset_index(inplace=True)
        temp.columns = ["Attribute", "Recent"]

        # add (ticker, dataframe) to main dictionary
        tickers_data[ticker] = temp

    return tickers_data
        
        

In [17]:
#yf.Ticker('1346.HK').info
#yf.Ticker('MPR.XA').info

In [29]:
if isolate_us:
    stock_data = get_stock_data (us)
else:
    stock_data = get_stock_data (b_corp_df)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SZL.XA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SZL.XA&crumb=4mfJ%2FGcyIwy


AttributeError: 'dict' object has no attribute 'head'

{'BIRD':              Attribute                 Recent
 0             address1  730 Montgomery Street
 1                 city          San Francisco
 2                state                     CA
 3                  zip                  94111
 4              country          United States
 ..                 ...                    ...
 112       grossMargins                0.41026
 113      ebitdaMargins               -0.38513
 114   operatingMargins               -0.40288
 115  financialCurrency                    USD
 116   trailingPegRatio                   None
 
 [117 rows x 2 columns],
 'AMAL':              Attribute              Recent
 0             address1  275 Seventh Avenue
 1                 city            New York
 2                state                  NY
 3                  zip               10001
 4              country       United States
 ..                 ...                 ...
 115     earningsGrowth              -0.058
 116      revenueGrowth               0.0

Attributes = ['address1',
 'city',
 'state',
 'zip',
 'country',
 'phone',
 'website',
 'industry',
 'industryKey',
 'industryDisp',
 'sector',
 'sectorKey',
 'sectorDisp',
 'longBusinessSummary',
 'fullTimeEmployees',
 'companyOfficers',
 'auditRisk',
 'boardRisk',
 'compensationRisk',
 'shareHolderRightsRisk',
 'overallRisk',
 'governanceEpochDate',
 'compensationAsOfEpochDate',
 'maxAge',
 'priceHint',
 'previousClose',
 'open',
 'dayLow',
 'dayHigh',
 'regularMarketPreviousClose',
 'regularMarketOpen',
 'regularMarketDayLow',
 'regularMarketDayHigh',
 'beta',
 'forwardPE',
 'volume',
 'regularMarketVolume',
 'averageVolume',
 'averageVolume10days',
 'averageDailyVolume10Day',
 'bid',
 'ask',
 'bidSize',
 'askSize',
 'marketCap',
 'fiftyTwoWeekLow',
 'fiftyTwoWeekHigh',
 'priceToSalesTrailing12Months',
 'fiftyDayAverage',
 'twoHundredDayAverage',
 'currency',
 'enterpriseValue',
 'profitMargins',
 'floatShares',
 'sharesOutstanding',
 'sharesShort',
 'sharesShortPriorMonth',
 'sharesShortPreviousMonthDate',
 'dateShortInterest',
 'sharesPercentSharesOut',
 'heldPercentInsiders',
 'heldPercentInstitutions',
 'shortRatio',
 'shortPercentOfFloat',
 'impliedSharesOutstanding',
 'bookValue',
 'priceToBook',
 'lastFiscalYearEnd',
 'nextFiscalYearEnd',
 'mostRecentQuarter',
 'netIncomeToCommon',
 'trailingEps',
 'forwardEps',
 'pegRatio',
 'enterpriseToRevenue',
 'enterpriseToEbitda',
 '52WeekChange',
 'SandP52WeekChange',
 'exchange',
 'quoteType',
 'symbol',
 'underlyingSymbol',
 'shortName',
 'longName',
 'firstTradeDateEpochUtc',
 'timeZoneFullName',
 'timeZoneShortName',
 'uuid',
 'messageBoardId',
 'gmtOffSetMilliseconds',
 'currentPrice',
 'targetHighPrice',
 'targetLowPrice',
 'targetMeanPrice',
 'targetMedianPrice',
 'recommendationMean',
 'recommendationKey',
 'numberOfAnalystOpinions',
 'totalCash',
 'totalCashPerShare',
 'ebitda',
 'totalDebt',
 'quickRatio',
 'currentRatio',
 'totalRevenue',
 'debtToEquity',
 'revenuePerShare',
 'returnOnAssets',
 'returnOnEquity',
 'freeCashflow',
 'operatingCashflow',
 'revenueGrowth',
 'grossMargins',
 'ebitdaMargins',
 'operatingMargins',
 'financialCurrency',
 'trailingPegRatio']

In [19]:
combined_data = pd.concat(stock_data) #turn from dict to df
combined_data = combined_data.reset_index()

del combined_data["level_1"] # clean up unnecessary column
combined_data.columns = ["Ticker", "Attribute", "Recent"] # update column names


In [61]:
attr_to_keep = ['shortName', 'longName', 'industry', 'sector','previousClose', 'sharesOutstanding', 'impliedSharesOutstanding',
               'marketCap',  'currency', 'country', 'website', 'enterpriseValue',
                'trailingEps', 'forwardEps', 'enterprisetoEbitda', 'ebitda', 'grossMargins', 'ebitdaMargins', 'operatingMargins']

In [62]:
filtered = combined_data[combined_data["Attribute"].isin(attr_to_keep)].reset_index()
del filtered["index"] # clean up unnecessary column


In [63]:
print("Number of Stocks:", len(filtered['Ticker'].unique()))
display(filtered['Ticker'].unique())

Number of Stocks: 68


array(['BIRD', 'AMAL', 'ARZZ3.SA', 'AEF.XA', 'BFT.WA', 'BIO.XA',
       'CWEB.TO', 'CLSA3.SA', 'MLMFI.PA', 'COUR', 'FLOW.TO', 'FREY.PA',
       'GFI.NZ', 'GROV', 'HAN.MC', 'HLZ.MC', 'INCLU.BR', 'IBAB.BR',
       'KPG.XA', 'KCT.AQ', 'KMD.NZ', 'LMND', 'LFG.XA', 'NTCO3.SA',
       '2897.TW', '1H2.SI', 'ALQP.PA', 'RETI.MI', '6088.T', 'SML.NZ',
       'COCO', 'VNT.MI', 'CONCHATORO.SN', 'VITL', 'VVPR', 'VHT.XA',
       'WRBY', 'ZVIA', 'ABB.XA', 'TIME.MI', 'CVT.NZ', 'DOX.MI',
       'SOMA3.SA', '1112.HK', 'HMGS.TA', '1346.HK', '0973.HK', 'MME.XA',
       'MPR.XA', 'TENG.L', 'TPX.L', 'XSG.L', 'XHS.MI', 'UL', 'ABI.BR',
       'PG', 'FFH.TO', 'BUD', 'KO', 'GPS', 'KIK.F', 'NESN.SW', 'OPY',
       '4755.T', 'HAIN', 'DANOY', 'CABK.MC', 'ISUN'], dtype=object)

In [64]:
display(filtered[filtered['Attribute'] == 'currency'])

,Ticker,Attribute,Recent
6,BIRD,currency,USD
24,AMAL,currency,USD
39,ARZZ3.SA,currency,BRL
54,AEF.XA,currency,AUD
69,BFT.WA,currency,PLN
...,...,...,...
1029,4755.T,currency,JPY
1047,HAIN,currency,USD
1065,DANOY,currency,USD
1082,CABK.MC,currency,EUR


In [65]:
#display(stock_data.head())
display(filtered.head())
stock_data_filtered = filtered.pivot(columns = 'Attribute', index='Ticker', values='Recent')
stock_data_filtered.columns
stock_data_filtered = stock_data_filtered[['shortName', 'longName','country', 'currency', 'previousClose',
                                           'sharesOutstanding', 'impliedSharesOutstanding','marketCap', 'enterpriseValue',
                                           'ebitda', 'forwardEps',  'trailingEps', 'grossMargins', 'operatingMargins', 'ebitdaMargins',
                                             'industry', 'sector', 'website']]

,Ticker,Attribute,Recent
0,BIRD,country,United States
1,BIRD,website,https://www.allbirds.com
2,BIRD,industry,Apparel Retail
3,BIRD,sector,Consumer Cyclical
4,BIRD,previousClose,0.6876


In [66]:
#stock_data_filtered.to_csv('Stock_data_24.03.25_a.csv')
stock_data_filtered

Attribute,shortName,longName,country,currency,previousClose,sharesOutstanding,impliedSharesOutstanding,marketCap,enterpriseValue,ebitda,forwardEps,trailingEps,grossMargins,operatingMargins,ebitdaMargins,industry,sector,website
Ticker,,,,,,,,,,,,,,,,,,
0973.HK,L'OCCITANE,L'Occitane International S.A.,Luxembourg,HKD,31.5,1472169984,1472169984,46888615936,47115563008,378692992,1.6,0.5,0.79602,0.08024,0.16421,Household & Personal Products,Consumer Defensive,https://group.loccitane.com
1112.HK,H&H INTL HLDG,Health and Happiness (H&H) International Holdi...,Hong Kong,HKD,11.24,645561024,645561024,7010792448,15227624448,2015838976,1.58,1.26,0.5996,0.15885,0.14606,Packaged Foods,Consumer Defensive,https://www.hh.global
1346.HK,LEVER STYLE,Lever Style Corporation,Hong Kong,HKD,0.95,639100032,689366016,600754048,579670272,18602392,NaN,0.19,0.28717,0.09904,0.08921,Apparel Manufacturing,Consumer Cyclical,https://www.leverstyle.com
1H2.SI,$ Olive Tree,Olive Tree Estates Limited,Singapore,SGD,0.078,115845000,124756000,9035910,13570926,-218000,NaN,-0.02,0.97498,-0.66778,-0.18182,Real Estate - Diversified,Real Estate,https://www.olivetreeestates.com
2897.TW,O-BANK CO LTD,"O-Bank Co., Ltd.",Taiwan,TWD,9.97,2716470016,2716470016,26920216576,107982684160,NaN,NaN,0.78,NaN,0.27909,NaN,Banks - Regional,Financial Services,https://www.o-bank.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VVPR,VivoPower International PLC,VivoPower International PLC,United Kingdom,USD,1.48,3270600,3270600,4742370,38970664,-5913000,-0.41,-8.2,0.10544,-0.49761,-0.39263,Solar,Technology,https://vivopower.com
WRBY,Warby Parker Inc.,Warby Parker Inc.,United States,USD,12.75,98374096,118163000,1520757760,1464138624,-33442000,0.28,-0.54,0.5453,-0.13326,-0.04993,Medical Instruments & Supplies,Healthcare,https://www.warbyparker.com
XHS.MI,XENIA HOTELLERIE SOLUTION,Xenia Hôtellerie Solution S.p.A. Società Benefit,Italy,EUR,2.66,3219000,3315810,8562540,20608540,2879812,NaN,NaN,0.09808,0.04408,0.0596,Lodging,Consumer Cyclical,https://www.xeniahs.com


In [24]:
# Get avaialable currencies
currency_dict = {}

url = "https://currency-conversion-and-exchange-rates.p.rapidapi.com/symbols"

headers = {"X-RapidAPI-Key": "af4b092441msh3f4304a0bce0366p1680a3jsn93d4c3a0659f",
           "X-RapidAPI-Host": "currency-conversion-and-exchange-rates.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

api_data = response.json()

    
for symbol, curr in api_data['symbols'].items():
    currency_dict[symbol] = (curr, 0)

#currency_dict
    

{'ANG': ('Netherlands Antillean Guilder', 0),
 'SVC': ('Salvadoran Colón', 0),
 'CAD': ('Canadian Dollar', 0),
 'XCD': ('East Caribbean Dollar', 0),
 'MVR': ('Maldivian Rufiyaa', 0),
 'HRK': ('Croatian Kuna', 0),
 'AUD': ('Australian Dollar', 0),
 'MWK': ('Malawian Kwacha', 0),
 'XAG': ('Silver (troy ounce)', 0),
 'MAD': ('Moroccan Dirham', 0),
 'PHP': ('Philippine Peso', 0),
 'NAD': ('Namibian Dollar', 0),
 'GNF': ('Guinean Franc', 0),
 'KES': ('Kenyan Shilling', 0),
 'MZN': ('Mozambican Metical', 0),
 'BTN': ('Bhutanese Ngultrum', 0),
 'MGA': ('Malagasy Ariary', 0),
 'AZN': ('Azerbaijani Manat', 0),
 'XAU': ('Gold (troy ounce)', 0),
 'RON': ('Romanian Leu', 0),
 'CHF': ('Swiss Franc', 0),
 'EGP': ('Egyptian Pound', 0),
 'BSD': ('Bahamian Dollar', 0),
 'TWD': ('New Taiwan Dollar', 0),
 'GGP': ('Guernsey Pound', 0),
 'LVL': ('Latvian Lats', 0),
 'MMK': ('Myanma Kyat', 0),
 'WST': ('Samoan Tala', 0),
 'ILS': ('Israeli New Sheqel', 0),
 'BHD': ('Bahraini Dinar', 0),
 'GBP': ('British Pou

In [25]:
## Access Exchange Rates:

url = "https://currency-conversion-and-exchange-rates.p.rapidapi.com/latest"

#querystring = {"from":"USD","to":"EUR,GBP"}
querystring = {'base':'USD'}

headers = {"X-RapidAPI-Key": "af4b092441msh3f4304a0bce0366p1680a3jsn93d4c3a0659f",
           "X-RapidAPI-Host": "currency-conversion-and-exchange-rates.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

exch_data = response.json()

for key in exch_data: 
    print(key,":", exch_data[key]) 

timestamp : 1711374185
base : USD
success : True
date : 2024-03-25
rates : {'ANG': 1.801965, 'SVC': 8.748603, 'CAD': 1.35769, 'XCD': 2.70255, 'MVR': 15.450579, 'HRK': 6.882375, 'AUD': 1.527931, 'MWK': 1733.546314, 'XAG': 0.040264, 'MAD': 10.074328, 'PHP': 56.212499, 'NAD': 19.009774, 'GNF': 8593.483098, 'KES': 132.240206, 'MZN': 63.503045, 'BTN': 83.369035, 'MGA': 4375.646329, 'AZN': 1.697463, 'XAU': 0.000459, 'RON': 4.587502, 'CHF': 0.896965, 'EGP': 47.400202, 'BSD': 0.999778, 'TWD': 31.835296, 'GGP': 0.793084, 'LVL': 0.60489, 'MMK': 2099.626056, 'WST': 2.749509, 'ILS': 3.636355, 'BHD': 0.376907, 'GBP': 0.790702, 'TZS': 2554.999589, 'SDG': 585.999926, 'LAK': 21056.470675, 'DJF': 178.043488, 'BYN': 3.271979, 'LBP': 89534.273896, 'RWF': 1284.243571, 'PEN': 3.703372, 'EUR': 0.922795, 'ZMK': 9001.196439, 'RSD': 108.155045, 'INR': 83.40895, 'MUR': 46.300075, 'BWP': 13.724205, 'GEL': 2.698534, 'KMF': 455.550616, 'UZS': 12595.218969, 'RUB': 92.134968, 'CUC': 1, 'BGN': 1.804801, 'JOD': 0.7088

## Upload Stock Universe Data after accounting for Exhange Rates

In [134]:
stocks_updated = pd.read_csv('Stock_data_24.03.25_updated.csv')
display(stocks_updated.head())

,Ticker,country,currency,Exchange Rate 03/25/2024),previousClose,previousCloseUSD,impliedSharesOutstanding,marketCapHomeCurrency,marketCapUSD
0,PG,United States,USD,$1.00,161.66,$161.66,2.353020e+09,3.78954E+11,3.789540e+11
1,NESN.SW,Switzerland,CHF,$1.11,94.36,$104.94,2.621310e+09,2.48107E+11,2.759200e+11
2,KO,United States,USD,$1.00,60.49,$60.49,4.312460e+09,2.61603E+11,2.616030e+11
3,UL,United Kingdom,USD,$1.00,49.99,$49.99,2.530770e+09,1.25592E+11,1.255920e+11
4,BUD,Belgium,USD,$1.00,59.50,$59.50,2.821060e+08,1.19174E+11,1.191740e+11


In [135]:
def get_price_history2 (df, start, end):
    tickers_list = df['Ticker'].tolist()
    tickers_data= {}
    first_object = yf.Ticker(tickers_list[0])
    prices = pd.DataFrame.from_dict(first_object.history(start=start, end=end, interval = "1d"))
    prices.drop(['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
    prices.columns = [tickers_list[0]]
    
    for ticker in tickers_list[1:]:
        ticker_object = yf.Ticker(ticker)
        try:
            temp = pd.DataFrame.from_dict(ticker_object.history(start=start, end=end, interval = "1d"))
            temp.drop(['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
            temp.columns = [ticker]
            prices = prices.merge(temp, how="outer", left_index=True, right_index=True)
        except:
            continue

    return prices

In [136]:
yf.Ticker("NESN.SW").history(start='2022-01-01', end='2022-01-25', interval='1d')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-01-03 00:00:00+01:00,122.214293,123.662425,122.099972,123.109848,3412729,0.0,0.0
2022-01-04 00:00:00+01:00,123.662420,123.662420,122.061851,123.376602,2920374,0.0,0.0
2022-01-05 00:00:00+01:00,122.138077,122.157135,119.718187,120.042107,5579892,0.0,0.0
2022-01-06 00:00:00+01:00,118.632082,119.908724,118.346272,119.775345,3806803,0.0,0.0
2022-01-07 00:00:00+01:00,119.394256,119.641964,118.174782,118.574921,3579390,0.0,0.0
2022-01-10 00:00:00+01:00,118.708299,118.727357,115.831103,116.135971,4560274,0.0,0.0
2022-01-11 00:00:00+01:00,116.478944,117.488820,115.964477,116.402725,3707378,0.0,0.0
2022-01-12 00:00:00+01:00,116.707593,116.860031,115.469060,116.040695,5324981,0.0,0.0
2022-01-13 00:00:00+01:00,116.040702,116.345571,115.316637,115.659615,4004810,0.0,0.0


In [137]:
prices = get_price_history2(stocks_updated, start='2022-01-01', end='2023-12-31')

KCT.AQ: Period '1mo' is invalid, must be one of ['1d', '5d']


In [138]:
print(prices.shape)

top20=prices.iloc[:,0:20]
print(top20.shape)
top30=prices.iloc[:,0:30]
print(top30.shape)

(4535, 67)
(4535, 20)
(4535, 30)


In [159]:
def calc_index2(df, num_in_index = 30, divisor = 1000000000):
    cols = df.columns
    mkt_cap_cols = []
    for col in cols:
        if "marketCapUSD" in col:
            mkt_cap_cols.append(col)
    
    mkt_cap_df = df[mkt_cap_cols].copy()
    
    mkt_cap_df = mkt_cap_df.iloc[0:num_in_index,:]
    #display(mkt_cap_df)
    
    # Sort by beginning mkt cap
    baseline_order = mkt_cap_df.iloc[0,:].sort_values(ascending=False)
    #baseline_all = baseline_order.index.to_list()

    
    mkt_cap_df['Total_Index'] = mkt_cap_df.sum(axis=0)/divisor
    
    return mkt_cap_df

In [160]:
calc_index2(stocks_updated)

,marketCapUSD,Total_Index
0,3.789540e+11,NaN
1,2.759200e+11,NaN
2,2.616030e+11,NaN
3,1.255920e+11,NaN
4,1.191740e+11,NaN
5,1.176780e+11,NaN
6,4.152268e+10,NaN
7,3.796921e+10,NaN
8,2.637475e+10,NaN
9,1.257604e+10,NaN


In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(3, 1, figsize = (20,18))
ax0.plot(normed_data0, color = 'seagreen', linewidth=2, label = 'With Subs')
ax0.legend(loc=3)
ax1.plot(normed_data1, color = 'royalblue', linewidth=2,label='Cert B Corps')
ax1.legend(loc=3)
ax2.plot(normed_data0, color = 'seagreen', linewidth=2, label = 'With Subs')
ax2.plot(normed_data1, color = 'royalblue', linewidth=2, label = 'Cert B Corps')
ax2.plot(normed_spx, color='darkorange', linewidth=2, label = 'SPX')
ax2.legend(loc=3)
plt.show()

In [105]:
## This is the old code.....

def get_price_history (df, start, end, add_subs = False):
    tickers_list = df['Ticker'].tolist()
    if add_subs:
        tickers_list = tickers_list + public_subs

    tickers_data= {}
    first_object = yf.Ticker(tickers_list[0])
    prices = pd.DataFrame.from_dict(first_object.history(start=start, end=end, interval = "1d"))
    prices.drop(['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
    prices.columns = [tickers_list[0]]
    
    for ticker in tickers_list[1:]:
        ticker_object = yf.Ticker(ticker)
        temp = pd.DataFrame.from_dict(ticker_object.history(start=start, end=end, interval = "1d"))
        temp.drop(['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
        temp.columns = [ticker]
        prices = prices.merge(temp, how="outer", left_index=True, right_index=True)

    return prices

In [104]:
#px_df = get_price_history(filtered, start="2022-01-01", end = "2024-02-23")
spx_object = yf.Ticker("^GSPC")
spx = pd.DataFrame.from_dict(spx_object.history(start="2022-01-01", end="2024-02-23", interval = "1d"))
spx

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-01-03 00:00:00-05:00,4778.140137,4796.640137,4758.169922,4796.560059,3831020000,0.0,0.0
2022-01-04 00:00:00-05:00,4804.509766,4818.620117,4774.270020,4793.540039,4683170000,0.0,0.0
2022-01-05 00:00:00-05:00,4787.990234,4797.700195,4699.439941,4700.580078,4887960000,0.0,0.0
2022-01-06 00:00:00-05:00,4693.390137,4725.009766,4671.259766,4696.049805,4295280000,0.0,0.0
2022-01-07 00:00:00-05:00,4697.660156,4707.950195,4662.740234,4677.029785,4181510000,0.0,0.0
...,...,...,...,...,...,...,...
2024-02-15 00:00:00-05:00,5003.140137,5032.720215,4999.439941,5029.729980,4137970000,0.0,0.0
2024-02-16 00:00:00-05:00,5031.129883,5038.700195,4999.520020,5005.569824,3833270000,0.0,0.0
2024-02-20 00:00:00-05:00,4989.319824,4993.709961,4955.020020,4975.509766,4034880000,0.0,0.0


In [ ]:
def add_shares_out (df, add_subs = False):
    df['BIRD_SharesOut'] = 152850000
    df['BIRD_MktCap'] = df['BIRD_SharesOut'] * df['BIRD']

    df['AMAL_SharesOut'] = 30430000
    df['AMAL_MktCap'] = df['AMAL_SharesOut'] * df['AMAL']

    df['COUR_SharesOut'] = 156950000
    df['COUR_MktCap'] = df['COUR_SharesOut'] * df['COUR']

    df['GROV_SharesOut'] = 37450000
    df['GROV_MktCap'] = df['GROV_SharesOut'] * df['GROV']

    df['LMND_SharesOut'] = 69920000
    df['LMND_MktCap'] = df['LMND_SharesOut'] * df['LMND']

    df['COCO_SharesOut'] = 56800000
    df['COCO_MktCap'] = df['COCO_SharesOut'] * df['COCO']

    df['VITL_SharesOut'] = 41600000
    df['VITL_MktCap'] = df['VITL_SharesOut'] * df['VITL']

    df['VVPR_SharesOut'] = 3270000
    df['VVPR_MktCap'] = df['VVPR_SharesOut'] * df['VVPR']

    df['WRBY_SharesOut'] = 117640000
    df['WRBY_MktCap'] = df['WRBY_SharesOut'] * df['WRBY']

    df['ZVIA_SharesOut'] = 50410000
    df['ZVIA_MktCap'] = df['ZVIA_SharesOut'] * df['ZVIA']


    if add_subs:
        df['UL_SharesOut'] = 2499020000
        df['UL_MktCap'] = df['UL_SharesOut'] * df['UL']

        df['PG_SharesOut'] = 2353020000
        df['PG_MktCap'] = df['PG_SharesOut'] * df['PG']

        df['BUD_SharesOut'] = 1986770000
        df['BUD_MktCap'] = df['BUD_SharesOut'] * df['BUD']

        df['KO_SharesOut'] = 4312460000
        df['KO_MktCap'] = df['KO_SharesOut'] * df['KO']


        df['GPS_SharesOut'] = 370830000 
        df['GPS_MktCap'] = df['GPS_SharesOut'] * df['GPS']


        df['OPY_SharesOut'] = 10290000
        df['OPY_MktCap'] = df['OPY_SharesOut'] * df['OPY']


        df['HAIN_SharesOut'] = 89830000
        df['HAIN_MktCap'] = df['HAIN_SharesOut'] * df['HAIN']


        df['DANOY_SharesOut'] = 2499020000
        df['DANOY_MktCap'] = df['DANOY_SharesOut'] * df['DANOY']
        
    col_order = df.columns.sort_values().tolist()
    df = df[col_order]
    
    return df


In [ ]:
#col_order = px_df.columns.sort_values().tolist()
#px_df = px_df[col_order]


In [ ]:
#px_df.to_csv('Public_B_Corps.csv')

In [ ]:
## This is the old code


def calc_index(df, divisor = 1000000000):
    cols = df.columns
    mkt_cap_cols = []
    for col in cols:
        if "MktCap" in col:
            mkt_cap_cols.append(col)
    
    mkt_cap_df = df[mkt_cap_cols].copy()
    
    # Sort by beginning mkt cap
    baseline_order = mkt_cap_df.iloc[0,:].sort_values(ascending=False)
    baseline_all = baseline_order.index.to_list()
    
    mkt_cap_df['Total_Index'] = mkt_cap_df[baseline_all].sum(axis=1)/divisor
    
    return mkt_cap_df

In [ ]:
def separate_subs (df, show_chart = True):

    px0 = get_price_history(df, start="2022-01-01", end = "2024-02-23", add_subs = True)
    px0 = add_shares_out(px0, add_subs = True)
    #px0.to_csv('24_03_19_BCorp_Index.csv')
    mkt_cap0 = calc_index(px0, divisor = 100000000)
    index0 = mkt_cap0[['Total_Index']]
    normed_data0 = index0.copy()
    normed_data0 = normed_data0.div(normed_data0.iloc[0]).fillna(0)


    px1 = get_price_history(df, start="2022-01-01", end = "2024-02-23", add_subs = False)
    px1 = add_shares_out(px1, add_subs = False)
    mkt_cap1 = calc_index(px1, divisor = 10000000)
    index1 = mkt_cap1[['Total_Index']]
    normed_data1 = index1.copy()
    normed_data1 = normed_data1.div(normed_data1.iloc[0]).fillna(0)
    
    # Add SPX:
    normed_spx = spx['Close'].copy()
    normed_spx = normed_spx.div(normed_spx.iloc[0]).fillna(0)
    
    if show_chart:
        fig, (ax0, ax1, ax2) = plt.subplots(3, 1, figsize = (20,18))
        ax0.plot(normed_data0, color = 'seagreen', linewidth=2, label = 'With Subs')
        ax0.legend(loc=3)
        ax1.plot(normed_data1, color = 'royalblue', linewidth=2,label='Cert B Corps')
        ax1.legend(loc=3)
        ax2.plot(normed_data0, color = 'seagreen', linewidth=2, label = 'With Subs')
        ax2.plot(normed_data1, color = 'royalblue', linewidth=2, label = 'Cert B Corps')
        ax2.plot(normed_spx, color='darkorange', linewidth=2, label = 'SPX')
        ax2.legend(loc=3)
        plt.show()
        
    return px0, px1

In [ ]:
all_px, bcorp_px = separate_subs(us, show_chart = True)

In [ ]:
all_px.to_csv("B_Corp_Stocks.csv")

## Construct a Subset - look at reweighting?

In [ ]:
def determine_reweighting (df, n=8):
    cols = df.columns
    mkt_cap_cols = []
    for col in cols:
        if "MktCap" in col:
            mkt_cap_cols.append(col)
    mkt_cap_df = df[mkt_cap_cols].copy()
    
    # Sort by beginning mkt cap
    baseline_order = mkt_cap_df.iloc[0,:].sort_values(ascending=False)
    baseline_all = baseline_order.index.to_list()
    baseline_n = baseline_order.index[0:n].to_list()
    
    #Add Quarterly Flag
    mkt_cap_df['Quarter'] = mkt_cap_df.index.quarter
    
    mkt_cap_df['Reweight'] = mkt_cap_df['Quarter'] - mkt_cap_df['Quarter'].shift(periods=1)
    
    quarterly = mkt_cap_df[mkt_cap_df['Reweight'].isin([1,-3])]
    
    for i in range(quarterly.shape[0]):
        order = quarterly.iloc[i,:].sort_values(ascending=False)
        order_all = order.index.to_list()
        order_n = order.index[0:n].to_list()
        for j in order_n:
            if j not in baseline_n:
                print("Need to Reweight!", i, j)
                print("Baseline n", baseline_n)
                print("New n", order_n)
        baseline_n = order_n
    
    return quarterly
    #mkt_cap_df['Total_Index'] = mkt_cap_df[baseline_all].sum(axis=1)/10000000
    #mkt_cap_df['Total_Index_8'] = mkt_cap_df[baseline_8].sum(axis=1)/10000000

               

In [ ]:
quarterly = determine_reweighting(us, n = 12)

### Visualization

#### Seaborn color maps:
‘Accent’, ‘Accent_r’, ‘Blues’, ‘Blues_r’, ‘BrBG’, ‘BrBG_r’, ‘BuGn’, ‘BuGn_r’, ‘BuPu’, ‘BuPu_r’, ‘CMRmap’, ‘CMRmap_r’, ‘Dark2’, ‘Dark2_r’, ‘GnBu’, ‘GnBu_r’, ‘Greens’, ‘Greens_r’, ‘Greys’, ‘Greys_r’, ‘OrRd’, ‘OrRd_r’, ‘Oranges’, ‘Oranges_r’, ‘PRGn’, ‘PRGn_r’, ‘Paired’, ‘Paired_r’, ‘Pastel1’, ‘Pastel1_r’, ‘Pastel2’, ‘Pastel2_r’, ‘PiYG’, ‘PiYG_r’, ‘PuBu’, ‘PuBuGn’, ‘PuBuGn_r’, ‘PuBu_r’, ‘PuOr’, ‘PuOr_r’, ‘PuRd’, ‘PuRd_r’, ‘Purples’, ‘Purples_r’, ‘RdBu’, ‘RdBu_r’, ‘RdGy’, ‘RdGy_r’, ‘RdPu’, ‘RdPu_r’, ‘RdYlBu’, ‘RdYlBu_r’, ‘RdYlGn’, ‘RdYlGn_r’, ‘Reds’, ‘Reds_r’, ‘Set1’, ‘Set1_r’, ‘Set2’, ‘Set2_r’, ‘Set3’, ‘Set3_r’, ‘Spectral’, ‘Spectral_r’, ‘Wistia’, ‘Wistia_r’, ‘YlGn’, ‘YlGnBu’, ‘YlGnBu_r’, ‘YlGn_r’, ‘YlOrBr’, ‘YlOrBr_r’, ‘YlOrRd’, ‘YlOrRd_r’, ‘afmhot’, ‘afmhot_r’, ‘autumn’, ‘autumn_r’, ‘binary’, ‘binary_r’, ‘bone’, ‘bone_r’, ‘brg’, ‘brg_r’, ‘bwr’, ‘bwr_r’, ‘cividis’, ‘cividis_r’, ‘cool’, ‘cool_r’, ‘coolwarm’, ‘coolwarm_r’, ‘copper’, ‘copper_r’, ‘crest’, ‘crest_r’, ‘cubehelix’, ‘cubehelix_r’, ‘flag’, ‘flag_r’, ‘flare’, ‘flare_r’, ‘gist_earth’, ‘gist_earth_r’, ‘gist_gray’, ‘gist_gray_r’, ‘gist_heat’, ‘gist_heat_r’, ‘gist_ncar’, ‘gist_ncar_r’, ‘gist_rainbow’, ‘gist_rainbow_r’, ‘gist_stern’, ‘gist_stern_r’, ‘gist_yarg’, ‘gist_yarg_r’, ‘gnuplot’, ‘gnuplot2’, ‘gnuplot2_r’, ‘gnuplot_r’, ‘gray’, ‘gray_r’, ‘hot’, ‘hot_r’, ‘hsv’, ‘hsv_r’, ‘icefire’, ‘icefire_r’, ‘inferno’, ‘inferno_r’, ‘jet’, ‘jet_r’, ‘magma’, ‘magma_r’, ‘mako’, ‘mako_r’, ‘nipy_spectral’, ‘nipy_spectral_r’, ‘ocean’, ‘ocean_r’, ‘pink’, ‘pink_r’, ‘plasma’, ‘plasma_r’, ‘prism’, ‘prism_r’, ‘rainbow’, ‘rainbow_r’, ‘rocket’, ‘rocket_r’, ‘seismic’, ‘seismic_r’, ‘spring’, ‘spring_r’, ‘summer’, ‘summer_r’, ‘tab10’, ‘tab10_r’, ‘tab20’, ‘tab20_r’, ‘tab20b’, ‘tab20b_r’, ‘tab20c’, ‘tab20c_r’, ‘terrain’, ‘terrain_r’, ‘turbo’, ‘turbo_r’, ‘twilight’, ‘twilight_r’, ‘twilight_shifted’, ‘twilight_shifted_r’, ‘viridis’, ‘viridis_r’, ‘vlag’, ‘vlag_r’, ‘winter’, ‘winter_r’